# Задание 1. Прямые методы решения линейных систем

In [24]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [25]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [arg for arg in argv] + ['$$'])
    display(Markdown(res))

def get_latex_column(arg: str, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {arg} with indeces form 1 to {n}"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def print_system_output(A: Matrix, s: str, B: Matrix, X: Matrix, n: int) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(rf'{s} \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


In [26]:
n = int(input()) # Размерность матрицы A
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_tex('A = ', A.tex(), ',~~', 'B = ', B.tex())
print_tex(A.tex(), get_latex_column('x', n), '=', B.tex())

$$ A =  \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} ,~~ B =  \begin{pmatrix} 9.292900 \\ 2.433482 \\ 7.732198 \\ 3.900604 \end{pmatrix} $$

$$ \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}
\end{pmatrix} = \begin{pmatrix} 9.292900 \\ 2.433482 \\ 7.732198 \\ 3.900604 \end{pmatrix} $$

### Решение методом Гаусса единственного деления

In [27]:
def SingleGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    A_ = A.copy()
    B_ = B.copy()
    X = [[0] for _ in range(n)]
    for k in range(n):
        for j in range(k+1, n):
            d = A_[j][k] / A_[k][k]
            for i in range(k, n):
                A_[j][i] -= d * A_[k][i]
            B_[j] -= d * B_[k][0]
    for k in range(n-1, -1, -1):
        d = 0
        for j in range(k, n):
            s = A_[k][j] * X[j][0]
            d += s
        X[k][0] = (B_[k][0] - d) / A_[k][k]
    return Matrix(np.array(X))


print_system_output(A, 'A', B, SingleGauss(A, B, n), n)


$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.226749 \\ -0.344390 \\ 0.166507 \\ 0.959058 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} \begin{pmatrix} 0.226749 \\ -0.344390 \\ 0.166507 \\ 0.959058 \end{pmatrix} = \begin{pmatrix} 9.292900 \\ 2.433482 \\ 7.732198 \\ 3.900604 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
9.29289991316131\\
2.4334820085705444\\
7.732198196205971\\
3.9006041760134025
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
9.29289991316131\\
2.4334820085705435\\
7.732198196205972\\
3.9006041760134043
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
8.881784197001252e-16\\
-8.881784197001252e-16\\
-1.7763568394002505e-15
\end{pmatrix} $$

### Решение методом Гаусса с выбором главного элемента

In [28]:
def ModifiedGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n):
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

print_system_output(A, 'A', B, ModifiedGauss(A, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.226749 \\ -0.344390 \\ 0.166507 \\ 0.959058 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} \begin{pmatrix} 0.226749 \\ -0.344390 \\ 0.166507 \\ 0.959058 \end{pmatrix} = \begin{pmatrix} 9.292900 \\ 2.433482 \\ 7.732198 \\ 3.900604 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
9.29289991316131\\
2.4334820085705444\\
7.732198196205971\\
3.9006041760134025
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
9.29289991316131\\
2.4334820085705435\\
7.732198196205971\\
3.900604176013403
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
8.881784197001252e-16\\
0.0\\
-4.440892098500626e-16
\end{pmatrix} $$

### Решение методом $LU$-разложения

In [29]:
def GetLU(A: Matrix, n: int) -> Tuple[Matrix, Matrix]:
    L = Matrix(np.zeros((n, n)))
    U = Matrix(np.identity(n))
    for i in range(n):
        for j in range(i, n):
            s = 0
            for k in range(i):
                s += L[j][k] * U[k][i]
            L[j][i] = A[j][i] - s
        for j in range(i, n):
            s = 0
            for k in range(i):
                s += L[i][k] * U[k][j]
            U[i][j] = (A[i][j] - s) / L[i][i]
    return L, U

L, U = GetLU(A, n)
print_tex('L = ', L.tex())
print_tex('U = ', U.tex())
print_tex(r'L \times U = ', (np.matmul(L, U)).tex(), r'\stackrel{?}{=}', A.tex(), '= A')

$$ L =  \begin{pmatrix} 8.696558 & 0 & 0 & 0 \\ 6.705774 & 2.011376 & 0 & 0 \\ 0.274170 & 4.475073 & -18.195751 & 0 \\ 2.088731 & 9.636526 & -43.193953 & -19.093893 \end{pmatrix} $$

$$ U =  \begin{pmatrix} 1 & 0.075188 & 0.083536 & 0.890256 \\ 0 & 1 & 4.691213 & -2.908489 \\ 0 & 0 & 1 & -1.203233 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

$$ L \times U =  \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} = A $$

In [30]:
Y_ = SingleGauss(L, B, n)
X_ = SingleGauss(U, Y_, n)
print_system_output(A, 'A', B, X_, n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.226749 \\ -0.344390 \\ 0.166507 \\ 0.959058 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} \begin{pmatrix} 0.226749 \\ -0.344390 \\ 0.166507 \\ 0.959058 \end{pmatrix} = \begin{pmatrix} 9.292900 \\ 2.433482 \\ 7.732198 \\ 3.900604 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
9.29289991316131\\
2.4334820085705444\\
7.732198196205971\\
3.9006041760134025
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
9.29289991316131\\
2.433482008570543\\
7.732198196205969\\
3.900604176013391
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
1.3322676295501878e-15\\
1.7763568394002505e-15\\
1.1546319456101628e-14
\end{pmatrix} $$

### Нахождение обратной к матрице $A$

In [31]:
def inverse(A: Matrix, n: int) -> Matrix:
    R = Matrix(np.zeros(shape=(n, 1)))
    A_ = Matrix(np.zeros(shape=(n, n)))
    for i in range(n):
        R[i][0] = 1.
        S = SingleGauss(A, R, n)
        for j in range(n):
            A_[j][i] = S[j][0]
        R[i][0] = 0
    return A_

A_ = inverse(A, n)
print_tex(r'A^{-1} = ', A_.tex())
print_tex(r'A \times A^{-1} = ', (np.matmul(A, A_)).tex())

def norm(M: Matrix) -> float:
    return max([sum([abs(M[i][j]) for i in range(n)]) for j in range(n)])

print_tex(r'||A||_1 * ||A^{-1}||_1 = ', str(norm(A) * norm(A_)))


$$ A^{-1} =  \begin{pmatrix} 0.096550 & 0.015701 & -0.112395 & 0.041115 \\ -0.027076 & -0.006154 & -0.082351 & 0.143299 \\ -0.058297 & 0.091364 & 0.094634 & -0.063017 \\ 0.028468 & -0.025690 & 0.124325 & -0.052373 \end{pmatrix} $$

$$ A \times A^{-1} =  \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

$$ ||A||_1 * ||A^{-1}||_1 =  9.802470238923407 $$

### Решение системы в случае $Cx=b~:~c_{11}=10^{-8}a_{11}, c_{ij}=a_{ij}~\forall i,j~:~(i,j)\ne(1,1)$

In [32]:
C = A.copy()
C[0][0] *= 1.e-8
print_tex('A=', A.tex())
print_tex('C=', C.tex())

$$ A= \begin{pmatrix} 8.696558 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} $$

$$ C= \begin{pmatrix} 0.000000 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} $$

In [33]:
print_system_output(C, 'C', B, SingleGauss(C, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 1.414083 \\ -0.677361 \\ -0.550414 \\ 1.309152 \end{pmatrix} $$

$$ C \times X^* =  \begin{pmatrix} 0.000000 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} \begin{pmatrix} 1.414083 \\ -0.677361 \\ -0.550414 \\ 1.309152 \end{pmatrix} = \begin{pmatrix} 9.292900 \\ 2.433482 \\ 7.732198 \\ 3.900604 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
9.29289991316131\\
2.4334820085705444\\
7.732198196205971\\
3.9006041760134025
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
9.29289991316131\\
2.433482020933691\\
7.732198191950056\\
3.9006042140696575
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
-1.2363146506544354e-08\\
4.25591473174336e-09\\
-3.8056255036167386e-08
\end{pmatrix} $$

In [34]:
print_system_output(C, 'C', B, ModifiedGauss(C, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 1.414083 \\ -0.677361 \\ -0.550414 \\ 1.309152 \end{pmatrix} $$

$$ C \times X^* =  \begin{pmatrix} 0.000000 & 0.653878 & 0.726478 & 7.742167 \\ 6.705774 & 2.515570 & 9.995966 & 0.119794 \\ 0.274170 & 4.495687 & 2.820670 & 9.122116 \\ 2.088731 & 9.793574 & 2.187522 & 6.710290 \end{pmatrix} \begin{pmatrix} 1.414083 \\ -0.677361 \\ -0.550414 \\ 1.309152 \end{pmatrix} = \begin{pmatrix} 9.292900 \\ 2.433482 \\ 7.732198 \\ 3.900604 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
9.29289991316131\\
2.4334820085705444\\
7.732198196205971\\
3.9006041760134025
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
9.292899913161309\\
2.4334820085705435\\
7.732198196205971\\
3.9006041760134016
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
1.7763568394002505e-15\\
8.881784197001252e-16\\
0.0\\
8.881784197001252e-16
\end{pmatrix} $$